# 针对专业人员的 TensorFlow 2.0 入门

将 Tensorflow 导入您的程序：

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

加载并准备 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(type(x_train), x_train.shape)
print(type(y_train), y_train.shape)

# 添加一个颜色信道维，Conv2D需要4维输入
x_train = x_train[..., tf.newaxis]  # 涉及计算图吗？
x_test = x_test[..., tf.newaxis]

print(x_train.shape, x_test.shape)

使用 `tf.data` 来将数据集切分为 batch 以及混淆数据集：

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

print(type(train_ds))
print(dir(train_ds))
print(train_ds)

使用 Keras [模型子类化（model subclassing）API] 构建 `tf.keras` 模型：

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')  # 没有指定输入形状
    self.flatten = Flatten()  # 特征展平
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

为训练选择优化器与损失函数：

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()  # 损失函数

optimizer = tf.keras.optimizers.Adam()  # 优化函数

选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果。

In [ ]:
# 模型性能指标
train_loss = tf.keras.metrics.Mean(name='train_loss')  # 计算平均损失
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

使用 `tf.GradientTape` 来训练模型。在一个训练步中，对给定的训练批数据，利用模型前向计算预测结果和相对于真实数据的损失，接着计算损失对模型参数的梯度，然后利用反向传播更新模型参数。
还可以根据预测结果和真实结果计算本训练步在训练数据上的训练效果，例如，精度。

In [ ]:
# 定义训练函数或计算图
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)  # 前向计算，预测
    loss = loss_object(labels, predictions)  # 根据预测和参考计算损失
    
  gradients = tape.gradient(loss, model.trainable_variables)  # 损失对参数的梯度
  # 基于梯度反向传播，更新参数
  optimizer.apply_gradients(zip(gradients, model.trainable_variables)) 

  train_loss(loss)  # 训练后模型效果
  train_accuracy(labels, predictions)

tf.function将python函数构建为计算图，提高执行效率。去掉tf.function，功能照样执行。
tf.GradientTape所定义的上下文中的操作被记录在tape上，通过倒带反向传播计算梯度。

测试模型：

In [ ]:
# 定义评价函数或计算图
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

执行训练和评价：

In [ ]:
EPOCHS = 2

for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  # 训练
  for images, labels in train_ds:  # 对每一批
    train_step(images, labels)

  # 评价
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

该图片分类器现在在此数据集上训练得到了接近 98% 的准确率（accuracy）。